<a href="https://colab.research.google.com/github/juanchoguillo/S3LAB1_Microservices/blob/main/S3LAB1_with_new_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

!pip install flask_restx

In [ ]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


In [ ]:
# To see the last rows of my data 
data.tail()

,url,phishing
39995,http://www.diaperswappers.com/forum/member.php...,0
39996,http://posting.bohemian.com/northbay/Tools/Ema...,0
39997,http://www.tripadvisor.jp/Hotel_Review-g303832...,0
39998,http://www.baylor.edu/content/services/downloa...,0
39999,http://www.phinfever.com/forums/viewtopic.php?...,0


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [ ]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

/var/folders/m_/7y4j70t107x_l3p7dkgn6qgm0000gn/T/ipykernel_4661/2862829481.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)


,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [ ]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

In [ ]:
X


,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,0,0,0,0,0,0,47,18,0,1
1,0,0,0,0,0,0,73,41,0,0
2,0,0,0,0,0,0,92,18,0,1
3,0,0,0,0,0,0,172,18,0,3
4,0,0,0,0,0,0,90,50,0,1
...,...,...,...,...,...,...,...,...,...,...
39995,0,0,1,0,0,0,71,22,0,1
39996,0,0,0,0,0,0,65,20,0,1
39997,0,0,0,1,0,0,92,18,0,0
39998,0,0,1,0,0,0,65,14,0,0


In [ ]:
y

0        1
1        1
2        1
3        1
4        1
        ..
39995    0
39996    0
39997    0
39998    0
39999    0
Name: phishing, Length: 40000, dtype: int64

## Entrenar y guardar el modelo

In [ ]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)
cross_val_score(clf, X, y, cv=10)

array([0.75975, 0.7525 , 0.745  , 0.754  , 0.76025, 0.75825, 0.75475,
       0.74875, 0.7675 , 0.75675])

In [ ]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

In [ ]:
import pandas as pd

# Define the keywords to search for
keywords = ['https', 'login', '.php', '.html', '@', 'sign']

# Define the new URL to analyze
new_url = 'http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php'

# Create a DataFrame with the new URL
new_data = pd.DataFrame({'url': [new_url]})

# Extract the domain from the URL
domain = new_url.split('/')[2]

# Create binary columns indicating if the URL contains the keywords
for keyword in keywords:
    new_data['keyword_' + keyword] = new_data['url'].str.contains(keyword).astype(int)

# Compute the length of the URL and the length of the domain
new_data['length'] = len(new_url) - 2
new_data['length_domain'] = len(domain)

# Create a binary column indicating if the URL is an IP address
new_data['isIP'] = int(domain.replace('.', '').isdigit())

# Count the number of 'com' in the URL
new_data['count_com'] = new_url.count('com')

# Preprocess the data
new_data = new_data.drop('url', axis=1)

# Predict the probability of the URL being malicious
proba = clf.predict_proba(new_data)[:, 1]

print(proba)


[0.74142757]


/Users/juancardona/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- length
- length_domain
Feature names seen at fit time, yet now missing:
- lenght
- lenght_domain

  warnings.warn(message, FutureWarning)


In [ ]:
def fishing_prob_clc(url: str):
    # Define the keywords to search for
    keywords = ['https', 'login', '.php', '.html', '@', 'sign']

    # Define the new URL to analyze
    new_url = url

    # Create a DataFrame with the new URL
    new_data = pd.DataFrame({'url': [new_url]})

    # Extract the domain from the URL
    domain = new_url.split('/')[2]

    # Create binary columns indicating if the URL contains the keywords
    for keyword in keywords:
        new_data['keyword_' + keyword] = new_data['url'].str.contains(keyword).astype(int)

    # Compute the length of the URL and the length of the domain
    new_data['length'] = len(new_url) - 2
    new_data['length_domain'] = len(domain)

    # Create a binary column indicating if the URL is an IP address
    new_data['isIP'] = int(domain.replace('.', '').isdigit())

    # Count the number of 'com' in the URL
    new_data['count_com'] = new_url.count('com')

    # Preprocess the data
    new_data = new_data.drop('url', axis=1)

    # Predict the probability of the URL being malicious
    proba = clf.predict_proba(new_data)[:, 1]
    
    return proba

In [ ]:
fishing_prob_clc('http://consultoriojuridico.co/pp/www.paypal.com/')


/Users/juancardona/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- length
- length_domain
Feature names seen at fit time, yet now missing:
- lenght
- lenght_domain

  warnings.warn(message, FutureWarning)


array([0.36948383])

In [ ]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'phishing_clf.pkl', compress=3)

['phishing_clf.pkl']

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [ ]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [ ]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'URL', 
    type=str, 
    required=True, 
    help='URL to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": fishing_prob_clc(args['URL'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://10.0.0.99:5000/ (Press CTRL+C to quit)
10.0.0.99 - - [23/Apr/2023 15:24:49] "GET / HTTP/1.1" 200 -
10.0.0.99 - - [23/Apr/2023 15:24:50] "GET /swagger.json HTTP/1.1" 200 -
/Users/juancardona/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- length
- length_domain
Feature names seen at fit time, yet now missing:
- lenght
- lenght_domain

  warnings.warn(message, FutureWarning)
10.0.0.99 - - [23/Apr/2023 15:25:15] "GET /predict/?URL=%27http%3A%2F%2Fwww.vipturismolondres.com%2Fcom.br%2F%3Fatendimento%3DCliente%26%2FLgSgkszm64%2FB8aNzHa8Aj.php%27 HTTP/1.1" 200 -
